In [1]:
import requests
import time
import pandas as pd
import numpy as np
import psycopg2
import sqlalchemy


In [2]:
host = "34.70.251.126"
port = 5432
user = "admin"
password = "root"
database= "umweltmonitoring"

In [3]:
#count  der Messwerte

# Verbindung zur Datenbank herstellen
connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
print("Verbindung zur Datenbank erfolgreich hergestellt.")

# Einen Cursor erstellen, um SQL-Abfragen auszuführen
cursor = connection.cursor()

# SQL SELECT-Abfrage
select_query = "SELECT * FROM value;"

# Abfrage ausführen
cursor.execute(select_query)

# Ergebnis abrufen
rows = cursor.fetchall()

# Ergebnis in ein Pandas DataFrame umwandeln
df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])

# Ergebnis anzeigen
print(df)

# Cursor und Verbindung schließen
cursor.close()
connection.close()


Verbindung zur Datenbank erfolgreich hergestellt.
      index                time  temperature  luftfeuchtigkeit   pm10  pm2_5  \
0         1 2024-05-15 16:32:38        18.30             99.90   3.40   3.10   
1         2 2024-05-15 16:32:39        17.90             75.63   9.45   8.20   
2         3 2024-05-15 16:32:40        22.90             55.90   3.23   2.93   
3         4 2024-05-15 16:32:40        17.24             69.30  10.77   3.38   
4         5 2024-05-15 16:32:41        26.30             30.70   0.00   0.00   
...     ...                 ...          ...               ...    ...    ...   
3435   3436 2024-05-18 18:30:02        18.64             65.58  10.77   3.38   
3436   3437 2024-05-18 18:30:03        19.70             46.40   0.00   0.00   
3437   3438 2024-05-18 18:30:03        18.40             99.90   6.33   3.97   
3438   3439 2024-05-18 18:30:04        14.28              0.00   0.00   0.00   
3439   3440 2024-05-18 18:30:05        17.29             63.17   6.85 

In [5]:
senseid_counts = df['senseid_fk'].value_counts()

# Ergebnis anzeigen
print(senseid_counts)

5ae4a726223bd80019a367a6    430
5dde9523ba7944001da4150e    430
5c08379b1c28f9001a3f580c    430
5b4d11485dc1ec001b5452c7    430
65e8d93acbf5700007f920ca    430
590e0b0a51d3460011c725c4    430
605f498077a88b001bba3dc0    430
64722d1c9be0580007f776d9    430
Name: senseid_fk, dtype: int64


### Messwerte der einzelnen Stationen in ein Dataframe speichern

In [7]:
#Messstationen definieren
boxes_dic = {
    "Mersch Luxembourg" : "5ae4a726223bd80019a367a6",
    "Ettlingen Deutschland" : "5dde9523ba7944001da4150e",
    "Stuttgart Deutschland" : "5c08379b1c28f9001a3f580c",
    "München Deutschland" : "5b4d11485dc1ec001b5452c7",
    "Berlin Deutschland" : "65e8d93acbf5700007f920ca",
    "Hamburg Deutschland" : "590e0b0a51d3460011c725c4",
    "Dortmund Deutschland" : "605f498077a88b001bba3dc0",
    "Jena Deutschland" : "64722d1c9be0580007f776d9"
}

In [4]:
# Erstelle ein leeres DataFrame mit Spaltennamen
df_messwerte = pd.DataFrame(columns=["time", "temperature","luftfeuchtigkeit", "pm10", "pm2_5", "senseid_fk"])

In [5]:

# Assuming df_messwerte is already defined as a DataFrame

new_data = []  # A list to collect new data
for box_id, box_value in boxes_dic.items():
    
    try:
        response = requests.get(f'https://api.opensensemap.org/boxes/{box_value}?format=json')
        box = response.json() 

        temperature_value = None
        pm10_value = None
        pm25_value = None
        rel_Luftfeuchte_value = None

        # Traverse through the 'sensors' list
        for sensor in box.get('sensors', []):
            # Check if the sensor is for Temperature
            if sensor.get('title') == 'Temperatur':
                # Access the temperature value
                temperature_value = sensor.get('lastMeasurement', {}).get('value')
            
            if sensor.get('title') == 'PM10':
                # Access the PM10 value
                pm10_value = sensor.get('lastMeasurement', {}).get('value')

            if sensor.get('title') == 'PM2.5':
                # Access the PM2.5 value
                pm25_value = sensor.get('lastMeasurement', {}).get('value')

            if sensor.get('title') == 'rel. Luftfeuchte':
                # Access the relative humidity value
                rel_Luftfeuchte_value = sensor.get('lastMeasurement', {}).get('value')
                
                break  # Exit the loop once the temperature is found
            
        # Append new data to the list
        new_data.append({"temperature": temperature_value, "time": time.strftime("%Y-%m-%d %H:%M:%S"),
                        "luftfeuchtigkeit": rel_Luftfeuchte_value, "pm10": pm10_value, "pm2_5": pm25_value,
                        "senseid_fk": box_value})
    
    except:
        print("FUCK YOU SENSEBOX NO WORKING TODAY" + box_value)
    
    # Convert the new data list to a DataFrame
new_df = pd.DataFrame(new_data)
    
    # Append the new DataFrame to the existing DataFrame
df_messwerte = pd.concat([df_messwerte, new_df], ignore_index=True)
df_messwerte = df_messwerte.fillna(0)

# Convert specific columns to float64, handling non-numeric values
columns_to_convert = ["temperature", "luftfeuchtigkeit", "pm10", "pm2_5"]
for column in columns_to_convert:
    df_messwerte[column] = pd.to_numeric(df_messwerte[column], errors='coerce')

df_messwerte = df_messwerte.round(2)


In [6]:
df_messwerte

,time,temperature,luftfeuchtigkeit,pm10,pm2_5,senseid_fk
0,2024-05-15 18:12:53,18.50,99.90,4.20,3.00,5ae4a726223bd80019a367a6
1,2024-05-15 18:12:54,18.32,73.25,7.80,7.10,5dde9523ba7944001da4150e
2,2024-05-15 18:12:54,23.30,55.60,2.83,2.63,5c08379b1c28f9001a3f580c
3,2024-05-15 18:12:54,17.24,69.25,10.77,3.38,5b4d11485dc1ec001b5452c7
4,2024-05-15 18:12:54,26.50,30.20,0.00,0.00,65e8d93acbf5700007f920ca
5,2024-05-15 18:12:54,28.90,37.00,4.87,1.50,590e0b0a51d3460011c725c4
6,2024-05-15 18:12:54,22.61,0.00,0.00,0.00,605f498077a88b001bba3dc0
7,2024-05-15 18:12:54,24.82,27.19,3.28,1.50,64722d1c9be0580007f776d9


In [7]:
df_messwerte["time"] = pd.to_datetime(df_messwerte["time"])

In [8]:
df_messwerte.dtypes

time                datetime64[ns]
temperature                float64
luftfeuchtigkeit           float64
pm10                       float64
pm2_5                      float64
senseid_fk                  object
dtype: object

In [9]:
import sqlalchemy

# Define your PostgreSQL connection string
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

# Create a SQLAlchemy engine
engine = sqlalchemy.create_engine(connection_string)

try:
    # Connect to the database using psycopg2
    connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
    print("Verbindung zur Datenbank erfolgreich hergestellt.")

    # Now you can proceed with inserting the DataFrame into the PostgreSQL database
    df_messwerte.to_sql("value", con=engine, if_exists="append", index=False)
    print("Daten erfolgreich in die Datenbank eingefügt.")
    
except Exception as e:
    print(f"Fehler beim Herstellen der Verbindung zur Datenbank oder Einfügen der Daten: {e}")


Verbindung zur Datenbank erfolgreich hergestellt.
Daten erfolgreich in die Datenbank eingefügt.


### Zweites DataFrame für Daten der Messstationen erstellen

In [10]:
# Erstelle ein leeres DataFrame mit Spaltennamen
df_stations = pd.DataFrame(columns=["Ort","Land","Lat", "Lon", "Hoehe", "senseID"])

In [11]:
new_data = []  # Eine leere Liste, um neue Daten zu sammeln
for box_id, box_value in boxes_dic.items():
    try:
        response = requests.get(f'https://api.opensensemap.org/boxes/{box_value}?format=json')
        box = response.json() 
    except:
        print("FUCK YOU SENSEBOX NO WORKING TODAY" + box_value)
    temperature_value = None
    lat = None
    lon = None
    hoehe = None
    pm10_value = None
    pm25_value = None
    rel_Luftfeuchte_value = None
    ort_Land = box_id.split()

    # Extrahiere Koordinaten
    current_location = box.get('currentLocation', {})
    if current_location:
        coordinates = current_location.get('coordinates')
        if coordinates:
            lon = coordinates[0]
            lat = coordinates[1]
            height = coordinates[2] if len(coordinates) >= 3 else None

        
    # Füge neue Daten zur Liste hinzu
    new_data.append({"Ort": ort_Land[0], "Land":ort_Land[1], "Lat": lat,"Lon":lon,"Hoehe":height, "senseID": box_value})
    
    # Konvertiere die neue Datenliste in ein DataFrame
new_df = pd.DataFrame(new_data)
    
    # Hänge das neue DataFrame an das bestehende DataFrame an
df_stations = pd.concat([df_stations, new_df], ignore_index=True)

FUCK YOU SENSEBOX NO WORKING TODAY590e0b0a51d3460011c725c4


In [12]:
df_stations

,Ort,Land,Lat,Lon,Hoehe,senseID
0,Mersch,Luxembourg,49.754663,6.105682,250.0,5ae4a726223bd80019a367a6
1,Ettlingen,Deutschland,48.926209,8.374409,4.0,5dde9523ba7944001da4150e
2,Stuttgart,Deutschland,48.779610,9.213723,280.0,5c08379b1c28f9001a3f580c
3,München,Deutschland,48.146546,11.576033,20.0,5b4d11485dc1ec001b5452c7
4,Berlin,Deutschland,52.511270,13.397356,45.0,65e8d93acbf5700007f920ca
5,Hamburg,Deutschland,52.511270,13.397356,45.0,590e0b0a51d3460011c725c4
6,Dortmund,Deutschland,51.497350,7.494606,NaN,605f498077a88b001bba3dc0
7,Jena,Deutschland,50.934561,11.589701,146.0,64722d1c9be0580007f776d9


### Daten von der DB lesen

In [17]:
try:
    # Connect to the database using psycopg2
    connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
    print("Verbindung zur Datenbank erfolgreich hergestellt.")

    # SQL-Abfrage ausführen
    query = "SELECT * FROM value"  
    cursor = connection.cursor()
    cursor.execute(query)
    results = cursor.fetchall()

    # DataFrame erstellen
    df = pd.DataFrame(results, columns=[desc[0] for desc in cursor.description])
    df = df.drop(columns=["index"])
    # Verbindung schließen
    connection.close()
    print("Daten erfolgreich geladen.")
    
except Exception as e:
    print(f"Fehler beim Herstellen der Verbindung zur Datenbank oder Einfügen der Daten: {e}")


Verbindung zur Datenbank erfolgreich hergestellt.
Daten erfolgreich geladen.


In [18]:
df

,time,temperature,luftfeuchtigkeit,pm10,pm2_5,senseid_fk
0,2024-05-15 16:32:38,18.30,99.90,3.40,3.10,5ae4a726223bd80019a367a6
1,2024-05-15 16:32:39,17.90,75.63,9.45,8.20,5dde9523ba7944001da4150e
2,2024-05-15 16:32:40,22.90,55.90,3.23,2.93,5c08379b1c28f9001a3f580c
3,2024-05-15 16:32:40,17.24,69.30,10.77,3.38,5b4d11485dc1ec001b5452c7
4,2024-05-15 16:32:41,26.30,30.70,0.00,0.00,65e8d93acbf5700007f920ca
...,...,...,...,...,...,...
2075,2024-05-17 14:10:03,17.39,70.37,10.77,3.38,5b4d11485dc1ec001b5452c7
2076,2024-05-17 14:10:03,17.60,43.50,0.00,0.00,65e8d93acbf5700007f920ca
2077,2024-05-17 14:10:04,38.90,30.00,2.53,1.73,590e0b0a51d3460011c725c4
2078,2024-05-17 14:10:05,13.50,0.00,0.00,0.00,605f498077a88b001bba3dc0


### plotly / dash (aus Übung)

In [48]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
import requests
from datetime import datetime

In [ ]:
# Initialize Dash app
app = dash.Dash(__name__)

# Create layout
app.layout = html.Div([
    # Graph component
    dcc.Graph(id='live-graph'),

    # Triggers periodically
    dcc.Interval(
        id='interval-component',
        interval=10_000,
        n_intervals=0
    )
])

# Initialize empty dataframe
df = pd.DataFrame(columns=['timestamp', 'value'])


# Define callback to update graph
@app.callback(
    Output('live-graph', 'figure'),
    [Input('interval-component', 'n_intervals')]
)
def update_graph(n):  # increment from intervals

    # Request API
    box_id = "616dd3698bd2c9001cc6d293"
    url = f"https://api.opensensemap.org/boxes/{box_id}?format=json"
    data = requests.get(url).json()
    value = data['sensors'][2]['lastMeasurement']['value']
    timestamp = datetime.now()

    global df
    df = pd.concat([df, pd.DataFrame([{'timestamp': timestamp, 'value': float(value)}])], ignore_index=True)

    # Plot data
    trace = go.Scatter(x=df['timestamp'], y=df['value'], mode='lines+markers')
    layout = go.Layout(title='Real-time Sensor Data', xaxis=dict(title='Timestamp'), yaxis=dict(title='Value'))
    fig = go.Figure(data=[trace], layout=layout)
    return fig


if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

### Random Forest Modell trainieren

In [19]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error


# 1. Convert the time column to datetime objects
df['time'] = pd.to_datetime(df['time'])

# 2. Feature Engineering with Lags
lags = [1, 2, 3]  # Lags for 10, 20, and 30 minutes
for lag in lags:
    df[f'Temperatur_lag_{lag*10}'] = df.groupby('senseid_fk')['temperature'].shift(lag)

# 3. Create target variables for each prediction horizon
for minutes in [10, 20, 30]:
    shift = minutes // 10  # Calculate the shift based on 10-minute intervals
    df[f'Temperatur_in_{minutes}_Minuten'] = df.groupby('senseid_fk')['temperature'].shift(-shift)

# 4. Remove rows with missing values (introduced by lagging and shifting)
df.dropna(inplace=True)


In [20]:
# 5. Define target variables and create a dictionary for models
target_variables = ['Temperatur_in_10_Minuten', 'Temperatur_in_20_Minuten', 'Temperatur_in_30_Minuten']
models = {}

# 6. Train and evaluate models for each target variable
for target in target_variables:
    # 6.1. Select features and target variable
    X = df.drop(target_variables + ["time", "senseid_fk"], axis=1)
    y = df[target]

    # 6.2. Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 6.3. Create and train the Random Forest model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # 6.4. Make predictions and evaluate
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    print(f"\n{target}:")
    print(f"  Mean Squared Error: {mse}")
    print(f"  Mean Absolute Error: {mae}")

    # 6.5. Store the trained model
    models[target] = model


Temperatur_in_10_Minuten:
  Mean Squared Error: 0.45367572713415544
  Mean Absolute Error: 0.2834032262135375

Temperatur_in_20_Minuten:
  Mean Squared Error: 0.658470633013489
  Mean Absolute Error: 0.3806042174652441

Temperatur_in_30_Minuten:
  Mean Squared Error: 0.8626496319008645
  Mean Absolute Error: 0.4876920320645823
